<a href="https://colab.research.google.com/github/Jefmamede/br-python-challenges/blob/master/Colorir_foto_PB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Colorindo uma foto antiga


In [2]:
# Importando as bibliotecas necessárias
import numpy as np
from google.colab.patches import cv2_imshow
import cv2


# Iniciando variáveis com o caminho dos arquivos
protxt = "model/colorization_deploy_v2.prototxt"
model = "model/colorization_release_v2.caffemodel"
points = "model/pts_in_hull.npy"
image = "images/arvore.jpg"

In [ ]:
# Usando a Open CV para ler o modelo através da função cv2.dnn.readNetFromCaffe
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe(protxt, model)

# O np.load() é usado porque o arquivo de ponto de cluster está no formato numpy. 
pts = np.load(points)

In [ ]:
# Tratando cada ponto como uma convolução 1×1 e carregar para o modelo
class8 = net.getLayerId("class8_ab")
conv8 = net.getLayerId("conv8_313_rh")
pts = pts.transpose().reshape(2, 313, 1, 1)
net.getLayer(class8).blobs = [pts.astype("float32")]
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype="float32")]

In [ ]:
# Tratando imagem usando a Open CV
image = cv2.imread(image)
scaled = image.astype("float32") / 255.0
lab = cv2.cvtColor(scaled, cv2.COLOR_BGR2LAB)

In [ ]:
# Na primeira linha, usamos cv2.imread para carregar a imagem. 
# Para o pré-processamento, na segunda linha fizemos um redimensionamento da intensidade dos pixels para o intervalo de [0,1]
# Na terceira, convertemos o modelo de RGB para LAB.
# 224×224, que é a dimensão exigida pelo modelo dos pesquisadores e depois vamos extrair o canal L
resized = cv2.resize(lab, (224, 224))
L = cv2.split(resized)[0]
L -= 50

In [ ]:
# O canal L será extraído como input para rede, para que ela seja capaz de prever os canais A e B.
'print("[INFO] colorizing image...")'
net.setInput(cv2.dnn.blobFromImage(L))
ab = net.forward()[0, :, :, :].transpose((1, 2, 0))

# resize para retornar ao tamanho original
ab = cv2.resize(ab, (image.shape[1], image.shape[0]))


In [ ]:
# Extraindo o canal L da imagem original e concatenando com os canais A e B, da predição, formando assim a imagem colorida.
L = cv2.split(lab)[0]
colorized = np.concatenate((L[:, :, np.newaxis], ab), axis=2)

# convertendo modelos, de LAB para RGB, e excluindo qualquer pixel que tenha ficado fora do intervalo especificado, [0,1]
colorized = cv2.cvtColor(colorized, cv2.COLOR_LAB2BGR)
colorized = np.clip(colorized, 0, 1)

# retornando as intensidades dos pixels ao intervalo [0,255], pois durante o pré-processamento foram divididos por 255 e agora estamos multiplicando por 255
colorized = (255 * colorized).astype("uint8")

In [ ]:
# Mostrando na tela as duas imagens a original e a colorida
cv2.imshow("Original", image)
cv2.imshow("Colorized", colorized)
cv2.waitKey(0)